<a href="https://colab.research.google.com/github/praveentn/hgwxx7/blob/master/%5Cnlp%5Csimilarity%5Cbert_sentence_similarity_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 3.3MB/s 
     |████████████████████████████████| 778kB 13.4MB/s 
     |████████████████████████████████| 890kB 33.0MB/s 
     |████████████████████████████████| 3.0MB 44.3MB/s 
     |████████████████████████████████| 1.1MB 41.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.3-cp36-none-any.whl size=97299 sha256=31e8b9dbca1fc959f48e24d237281a05adcad7739f5450f8436809af5d7e736f
  Stored in directory: /root/.cache/pip/wheels/75/d6/0a/cab163b21d0597cc1580bc344487b11ad405e0d1d314725f2b
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=fe1bae775844ba9c13c8a15cf80762146c45fb641ef3fee8249078f35978e4c1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
!git clone https://github.com/UKPLab/sentence-transformers.git

Cloning into 'sentence-transformers'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 1657 (delta 69), reused 107 (delta 49), pack-reused 1506
Receiving objects: 100% (1657/1657), 511.68 KiB | 8.25 MiB/s, done.
Resolving deltas: 100% (1122/1122), done.


In [3]:
#pip install -e .

In [4]:
import os

os.listdir()

['.config', 'sentence-transformers', 'sample_data']

In [5]:
cd sentence-transformers/

/content/sentence-transformers


In [6]:
ls

docs/      LICENSE     README.md         sentence_transformers/  setup.py
examples/  NOTICE.txt  requirements.txt  setup.cfg               tests/


In [7]:
pip install -e .

Obtaining file:///content/sentence-transformers
  Found existing installation: sentence-transformers 0.3.3
    Uninstalling sentence-transformers-0.3.3:
      Successfully uninstalled sentence-transformers-0.3.3
  Running setup.py develop for sentence-transformers


In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:24<00:00, 16.5MB/s]


In [9]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)


In [ ]:
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

### Semantic Search

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'A cheetah is running behind its prey.'
          ]
corpus_embeddings = embedder.encode(corpus,) # convert_to_tensor=True)

In [48]:
type(corpus_embeddings), corpus_embeddings.shape

(numpy.ndarray, (9, 768))

In [43]:
# Query sentences:
queries = ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']

In [44]:
type(corpus)

list

In [45]:
corpus

['A man is eating food.',
 'A man is eating a piece of bread.',
 'The girl is carrying a baby.',
 'A man is riding a horse.',
 'A woman is playing violin.',
 'Two men pushed carts through the woods.',
 'A man is riding a white horse on an enclosed ground.',
 'A monkey is playing drums.',
 'A cheetah is running behind its prey.']

In [47]:
query_embedding = embedder.encode(queries[0])
type(query_embedding), query_embedding.shape

(numpy.ndarray, (768,))

In [50]:
import numpy as np
from scipy.spatial import distance

distances = distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]
min_index = np.argmin(distances)
min_distance = distances[min_index]
max_similarity = 1 - min_distance

max_similarity

0.5777363511703955

In [51]:
distances = distance.cdist([query_embedding], corpus_embeddings, "cosine")
distances

array([[0.42226365, 0.50136387, 1.29761573, 0.84193823, 1.00561219,
        0.90084603, 0.85258679, 0.95155343, 0.96996769]])

In [52]:
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query) #, convert_to_tensor=True)
    
    distances = distance.cdist([query_embedding], corpus_embeddings, "cosine")

    for d in list(distances):
        print(d)

[0.42226365 0.50136387 1.29761573 0.84193823 1.00561219 0.90084603
 0.85258679 0.95155343 0.96996769]
[0.87602063 0.82807884 1.10474561 0.9906584  0.9900898  0.97112727
 0.92935774 0.35648123 0.96478641]
[0.94020165 1.03033926 1.03776777 0.78840067 1.01113582 0.91313559
 0.75149814 0.81799566 0.2230935 ]


In [55]:
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query) #, convert_to_tensor=True)
    
    print("cosine scores ==> ",cosine_similarity([query_embedding], corpus_embeddings))

cosine scores ==>  [[ 0.5777364   0.49863607 -0.2976157   0.15806173 -0.00561219  0.09915397
   0.1474132   0.04844657  0.0300323 ]]
cosine scores ==>  [[ 0.1239794   0.17192118 -0.1047456   0.00934159  0.00991021  0.02887273
   0.07064228  0.64351887  0.03521359]]
cosine scores ==>  [[ 0.05979835 -0.03033926 -0.03776776  0.2115993  -0.01113583  0.0868644
   0.24850187  0.18200433  0.7769065 ]]


In [40]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query) #, convert_to_tensor=True)
    
    cos_scores = cosine_similarity(query_embedding.reshape(-1,1), corpus_embeddings.reshape(-1,1))

    
    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(idx)
        #print(np.array(corpus[idx]).strip(), "(Score: %.4f)" % (np.array(cos_scores[idx])))







Query: A man is eating pasta.

Top 5 most similar sentences in corpus:
[   0    1    2 ... 6909 6910 6911]
[   4   10   12 ... 6909 6910 6911]
[   0    1    2 ... 6909 6910 6911]
[   0    1    2 ... 6909 6910 6911]
[   4   10   12 ... 6909 6910 6911]




Query: Someone in a gorilla costume is playing a set of drums.

Top 5 most similar sentences in corpus:
[   0    1    2 ... 6909 6910 6911]
[   0    1    2 ... 6909 6910 6911]
[   0    1    2 ... 6909 6910 6911]
[   4   10   12 ... 6909 6910 6911]
[   0    1    2 ... 6909 6910 6911]




Query: A cheetah chases prey on across a field.

Top 5 most similar sentences in corpus:
[   4   10   12 ... 6909 6910 6911]
[   0    1    2 ... 6909 6910 6911]
[   4   10   12 ... 6909 6910 6911]
[   0    1    2 ... 6909 6910 6911]
[   4   10   12 ... 6909 6910 6911]


In [ ]:
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))

In [11]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import pickle
import time

model_name = 'distilbert-base-nli-stsb-quora-ranking'
model = SentenceTransformer(model_name)



100%|██████████| 245M/245M [00:15<00:00, 15.7MB/s]


In [12]:
url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
dataset_path = "quora_duplicate_questions.tsv"
max_corpus_size = 100000


embedding_cache_path = 'quora-embeddings-{}-size-{}.pkl'.format(model_name.replace('/', '_'), max_corpus_size)




In [13]:
#Check if embedding cache path exists
if not os.path.exists(embedding_cache_path):
    # Check if the dataset exists. If not, download and extract
    # Download dataset if needed
    if not os.path.exists(dataset_path):
        print("Download dataset")
        util.http_get(url, dataset_path)

    # Get all unique sentences from the file
    corpus_sentences = set()
    with open(dataset_path, encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            corpus_sentences.add(row['question1'])
            if len(corpus_sentences) >= max_corpus_size:
                break

            corpus_sentences.add(row['question2'])
            if len(corpus_sentences) >= max_corpus_size:
                break

    corpus_sentences = list(corpus_sentences)
    print("Encode the corpus. This might take a while")
    corpus_embeddings = model.encode(corpus_sentences, show_progress_bar=True, convert_to_tensor=True)

    print("Store file on disc")
    with open(embedding_cache_path, "wb") as fOut:
        pickle.dump({'sentences': corpus_sentences, 'embeddings': corpus_embeddings}, fOut)
else:
    print("Load pre-computed embeddings from disc")
    with open(embedding_cache_path, "rb") as fIn:
        cache_data = pickle.load(fIn)
        corpus_sentences = cache_data['sentences'][0:max_corpus_size]
        corpus_embeddings = cache_data['embeddings'][0:max_corpus_size]



Download dataset


100%|██████████| 58.2M/58.2M [00:00<00:00, 81.6MB/s]


Encode the corpus. This might take a while



Store file on disc


In [14]:
###############################
print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

while True:
    inp_question = input("Please enter a question: ")

    start_time = time.time()
    question_embedding = model.encode(inp_question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings)
    end_time = time.time()
    hits = hits[0]  #Get the hits for the first query

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))

    print("\n\n========\n")

Corpus loaded with 100000 sentences / embeddings
Please enter a question: What is Quantum Mechanics?
Input question: What is Quantum Mechanics?
Results (after 0.023 seconds):
	0.828	What is a Quantum?
	0.789	What is systems theory?
	0.789	What's a simple way of explaining quantum physics?
	0.784	What should I read to start learning about quantum mechanics?
	0.774	What is contract theory?



Please enter a question: 
Input question: 
Results (after 0.017 seconds):
	0.845	?
	0.818	.
	0.655	What is something that I don't know and you know?
	0.644	What is something that you know that I don't?
	0.641	Is it possible



Please enter a question: exit
Input question: exit
Results (after 0.018 seconds):
	0.694	How do I exit out of Quora?
	0.652	What impact are you wanting to leave on this world?
	0.616	How do I get out of Quora permanently?
	0.613	How do get out of Quora?
	0.611	How can I get out of this thing called Quora?



Please enter a question: !
Input question: !
Results (after 0.016 sec

KeyboardInterrupt: ignored